# Lane Navigation Training
for self driving car

In [3]:
# config

OUTPUT_DIR = 'lane_nav_output'
DATASET_DIR = 'lane_nav_data'

In [4]:
# imports

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

import cv2

import os

print(f"tensorflow version: {tf.__version__}")
print(f"keras version: {keras.__version__}")

# make output directory if needed
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

tensorflow version: 2.16.2
keras version: 3.8.0


In [24]:
# The model
def lane_nav_model():
    model = keras.layers.Sequential(name='lane_nav_model')

    # Input
    model.add(keras.layers.Input(shape=(200, 66, 1), batch_size=32))
    
    # Depthwise Separable Convolution for efficiency
    model.add(keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), depth_multiplier=1, padding='same'))
    model.add(keras.layers.Conv2D(24, (1, 1), activation=None))  # Pointwise Conv
    model.add(keras.layers.ReLU())
    
    model.add(keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), depth_multiplier=1, padding='same'))
    model.add(keras.layers.Conv2D(36, (1, 1), activation=None))
    model.add(keras.layers.ReLU())
    
    model.add(keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), depth_multiplier=1, padding='same'))
    model.add(keras.layers.Conv2D(48, (1, 1), activation=None))
    model.add(keras.layers.ReLU())
    
    model.add(keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same'))
    model.add(keras.layers.Conv2D(64, (1, 1), activation=None))
    model.add(keras.layers.ReLU())
    
    model.add(keras.layers.Dropout(0.2))  # Regularization
    
    model.add(keras.layers.DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same'))
    model.add(keras.layers.Conv2D(64, (1, 1), activation=None))
    model.add(keras.layers.ReLU())
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(100, activation='relu'))
    model.add(keras.layers.Dense(50, activation='relu'))
    model.add(keras.layers.Dense(10, activation='relu'))
    model.add(keras.layers.Dense(1))  # Steering angle output
    
    optimizer = keras.optimizers.Adam(learning_rate=1e-3)
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

In [25]:
model = nvidia_model()
model.summary()

Model: "Nvidia_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ depthwise_conv2d_10             │ (32, 100, 33, 1)       │            10 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (32, 100, 33, 24)      │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_10 (ReLU)                 │ (32, 100, 33, 24)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_11             │ (32, 50, 17, 24)       │           240 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (32, 50, 17, 36)       │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (32, 50, 17, 36)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_12             │ (32, 25, 9, 36)        │           360 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (32, 25, 9, 48)        │         1,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (32, 25, 9, 48)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_13             │ (32, 25, 9, 48)        │           480 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (32, 25, 9, 64)        │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (32, 25, 9, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (32, 25, 9, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ depthwise_conv2d_14             │ (32, 25, 9, 64)        │           640 │
│ (DepthwiseConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (32, 25, 9, 64)        │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (32, 25, 9, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (32, 14400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (32, 14400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (32, 100)              │     1,440,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (32, 50)               │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (32, 10)               │           510 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (32, 1)                │            11 │
└─────────────────────────────────┴────────────────────────┴─────────────

 Total params: 1,457,421 (5.56 MB)

 Trainable params: 1,457,421 (5.56 MB)

 Non-trainable params: 0 (0.00 B)